# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7ff838bd7fd0>
├── 'a' --> tensor([[-0.2875, -0.6631,  1.1169],
│                   [ 0.8940, -0.5351, -0.4906]])
└── 'x' --> <FastTreeValue 0x7ff838be2190>
    └── 'c' --> tensor([[ 1.3298, -0.2681, -1.3537,  0.1432],
                        [ 0.5488, -0.2302,  1.0011,  0.7344],
                        [-0.1489, -1.1674, -0.3327, -0.5693]])

In [4]:
t.a

tensor([[-0.2875, -0.6631,  1.1169],
        [ 0.8940, -0.5351, -0.4906]])

In [5]:
%timeit t.a

64.6 ns ± 0.0251 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7ff838bd7fd0>
├── 'a' --> tensor([[ 0.2024,  0.5615, -0.6237],
│                   [-0.5319, -1.0864,  0.2323]])
└── 'x' --> <FastTreeValue 0x7ff838be2190>
    └── 'c' --> tensor([[ 1.3298, -0.2681, -1.3537,  0.1432],
                        [ 0.5488, -0.2302,  1.0011,  0.7344],
                        [-0.1489, -1.1674, -0.3327, -0.5693]])

In [7]:
%timeit t.a = new_value

70.3 ns ± 0.0863 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.2875, -0.6631,  1.1169],
               [ 0.8940, -0.5351, -0.4906]]),
    x: Batch(
           c: tensor([[ 1.3298, -0.2681, -1.3537,  0.1432],
                      [ 0.5488, -0.2302,  1.0011,  0.7344],
                      [-0.1489, -1.1674, -0.3327, -0.5693]]),
       ),
)

In [10]:
b.a

tensor([[-0.2875, -0.6631,  1.1169],
        [ 0.8940, -0.5351, -0.4906]])

In [11]:
%timeit b.a

55.5 ns ± 0.0519 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.4030, -1.5260, -0.3330],
               [-0.1350,  0.7413, -2.6233]]),
    x: Batch(
           c: tensor([[ 1.3298, -0.2681, -1.3537,  0.1432],
                      [ 0.5488, -0.2302,  1.0011,  0.7344],
                      [-0.1489, -1.1674, -0.3327, -0.5693]]),
       ),
)

In [13]:
%timeit b.a = new_value

476 ns ± 0.0508 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

908 ns ± 1.12 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.6 µs ± 27.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 1.46 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 258 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7ff78d8aa280>
├── 'a' --> tensor([[[-0.2875, -0.6631,  1.1169],
│                    [ 0.8940, -0.5351, -0.4906]],
│           
│                   [[-0.2875, -0.6631,  1.1169],
│                    [ 0.8940, -0.5351, -0.4906]],
│           
│                   [[-0.2875, -0.6631,  1.1169],
│                    [ 0.8940, -0.5351, -0.4906]],
│           
│                   [[-0.2875, -0.6631,  1.1169],
│                    [ 0.8940, -0.5351, -0.4906]],
│           
│                   [[-0.2875, -0.6631,  1.1169],
│                    [ 0.8940, -0.5351, -0.4906]],
│           
│                   [[-0.2875, -0.6631,  1.1169],
│                    [ 0.8940, -0.5351, -0.4906]],
│           
│                   [[-0.2875, -0.6631,  1.1169],
│                    [ 0.8940, -0.5351, -0.4906]],
│           
│                   [[-0.2875, -0.6631,  1.1169],
│                    [ 0.8940, -0.5351, -0.4906]]])
└── 'x' --> <FastTreeValue 0x7ff78d8aa220>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.3 µs ± 82.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7ff79452d910>
├── 'a' --> tensor([[-0.2875, -0.6631,  1.1169],
│                   [ 0.8940, -0.5351, -0.4906],
│                   [-0.2875, -0.6631,  1.1169],
│                   [ 0.8940, -0.5351, -0.4906],
│                   [-0.2875, -0.6631,  1.1169],
│                   [ 0.8940, -0.5351, -0.4906],
│                   [-0.2875, -0.6631,  1.1169],
│                   [ 0.8940, -0.5351, -0.4906],
│                   [-0.2875, -0.6631,  1.1169],
│                   [ 0.8940, -0.5351, -0.4906],
│                   [-0.2875, -0.6631,  1.1169],
│                   [ 0.8940, -0.5351, -0.4906],
│                   [-0.2875, -0.6631,  1.1169],
│                   [ 0.8940, -0.5351, -0.4906],
│                   [-0.2875, -0.6631,  1.1169],
│                   [ 0.8940, -0.5351, -0.4906]])
└── 'x' --> <FastTreeValue 0x7ff838be2040>
    └── 'c' --> tensor([[ 1.3298, -0.2681, -1.3537,  0.1432],
                        [ 0.5488, -0.2302,  1.0011,  0.7344],
                 

In [23]:
%timeit t_cat(trees)

30.5 µs ± 89.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.2 µs ± 422 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.2875, -0.6631,  1.1169],
                [ 0.8940, -0.5351, -0.4906]],
       
               [[-0.2875, -0.6631,  1.1169],
                [ 0.8940, -0.5351, -0.4906]],
       
               [[-0.2875, -0.6631,  1.1169],
                [ 0.8940, -0.5351, -0.4906]],
       
               [[-0.2875, -0.6631,  1.1169],
                [ 0.8940, -0.5351, -0.4906]],
       
               [[-0.2875, -0.6631,  1.1169],
                [ 0.8940, -0.5351, -0.4906]],
       
               [[-0.2875, -0.6631,  1.1169],
                [ 0.8940, -0.5351, -0.4906]],
       
               [[-0.2875, -0.6631,  1.1169],
                [ 0.8940, -0.5351, -0.4906]],
       
               [[-0.2875, -0.6631,  1.1169],
                [ 0.8940, -0.5351, -0.4906]]]),
    x: Batch(
           c: tensor([[[ 1.3298, -0.2681, -1.3537,  0.1432],
                       [ 0.5488, -0.2302,  1.0011,  0.7344],
                       [-0.1489, -1.1674, -0.3327, -0.5693]],
         

In [26]:
%timeit Batch.stack(batches)

78.2 µs ± 245 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.2875, -0.6631,  1.1169],
               [ 0.8940, -0.5351, -0.4906],
               [-0.2875, -0.6631,  1.1169],
               [ 0.8940, -0.5351, -0.4906],
               [-0.2875, -0.6631,  1.1169],
               [ 0.8940, -0.5351, -0.4906],
               [-0.2875, -0.6631,  1.1169],
               [ 0.8940, -0.5351, -0.4906],
               [-0.2875, -0.6631,  1.1169],
               [ 0.8940, -0.5351, -0.4906],
               [-0.2875, -0.6631,  1.1169],
               [ 0.8940, -0.5351, -0.4906],
               [-0.2875, -0.6631,  1.1169],
               [ 0.8940, -0.5351, -0.4906],
               [-0.2875, -0.6631,  1.1169],
               [ 0.8940, -0.5351, -0.4906]]),
    x: Batch(
           c: tensor([[ 1.3298, -0.2681, -1.3537,  0.1432],
                      [ 0.5488, -0.2302,  1.0011,  0.7344],
                      [-0.1489, -1.1674, -0.3327, -0.5693],
                      [ 1.3298, -0.2681, -1.3537,  0.1432],
                      [ 0.5488, -

In [28]:
%timeit Batch.cat(batches)

144 µs ± 396 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

308 µs ± 1.32 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
